In [129]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker


In [130]:
engine = create_engine('sqlite:///iptables.db')
# connection = engine.connect()

Base = declarative_base()

Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [131]:
class IpStatus(Base):
    __tablename__ = "t_ipstatus"
    __table_args__ = {'extend_existing': True}
    Ip            = Column(String(30), primary_key=True)
    Status        = Column(String(30))
    City          = Column(String(30))
    ServerHall    = Column(String(30))
    ServerHallId  = Column(Integer)
    Env           = Column(String(30))
    RackNo        = Column(Integer)
    Remark        = Column(String(30))
    
    def __repr__(self):
        return "<IpStatus ( ip = '%s', Status = '%s', City = '%s', Serverhall = '%s', ServerHalId = '%s', Env = '%s', RackNo = '%s', Remark = '%s')>" % (
        self.Ip, self.Status, self.City, self.ServerHall, self.ServerHallId, self.Env, self.RackNo, self.Remark)
    

# 1. C - From CURD Operation

In [132]:
### Create a record function 

def create_record(ObjectType):
    output = None
    err = None
    
    try :
        session.add(ObjectType)
        session.commit()
    except Exception as e:
        print(e)
        session.rollback()

ip1 =IpStatus(Ip="10.144.98.151", Status="Free", City="NVMB",ServerHall="DC1", ServerHallId=10)
ip2 =IpStatus(Ip="10.144.98.152", Status="Free", City="NVMB",ServerHall="DC1", ServerHallId=10)
ip3 =IpStatus(Ip="10.144.98.153", Status="Free", City="JMNG",ServerHall="DC1", ServerHallId=10)
ip4 =IpStatus(Ip="10.144.98.154", Status="Free", City="JMNG",ServerHall="DC1", ServerHallId=10)
ip1.__table__.create(engine, checkfirst=True)
create_record(ip1)
create_record(ip2)
create_record(ip3)
create_record(ip4)

In [127]:
session.rollback()

# 2. R - From CURD Operation

<h1>Fetch a single record </h1>

In [6]:
#def read_record(ObjectType, whatToUpdate, valuesToUpdate):
def read_record():
    return session.query(IpStatus).first()
    
first_record = read_record()
print(first_record)

<IpStatus ( ip = '10.144.98.151', Status = 'Free', City = 'NVMB', Serverhall = 'DC1', ServerHalId = '10', Env = 'dev', RackNo = '2', Remark = '')>


<h1> Fetching records by giving dynamic flters </h1>

In [18]:
## If we are expecting multiple values for a key, we can use in as shown in line no 11. 
def read_record_dynamic_filters(filters):
    q = session.query(IpStatus)
    for k, v in filters.items():
        f = getattr(IpStatus,k)
        print(f)
#         q = q.filter(f.in_(v))
        q = q.filter(f == v)
    return q.all()

# f = {"Status": ["Free"], "City":["NVMB"]}
f = {"Status": "Free", "City":"NVMB"}
dynamic_filter = read_record_dynamic_filters(f)
print(len(dynamic_filter))

IpStatus.Status
5


<h1> Fetching Multiple values based on a condition </h1>

In [118]:
## Dynamic way to fetch the no of columns
def read_dynamic_columns(cols):
    q = session.query(*[getattr(IpStatus, attr) for attr in cols])
    return q.all()

f = ['Ip','Status','City', 'Env']
dynamic_column = read_dynamic_columns(f)
print(dynamic_column)


[('10.144.98.151', 'Used', 'NVMB', 'dev'), ('10.144.98.152', 'Used', 'NVMB', 'dev'), ('10.144.98.153', 'Used', 'NVMB', None), ('10.144.98.154', 'Used', 'NVMB', None), ('10.144.98.155', 'Used', 'JMNG', None)]


# Combining all the conditions to make a generic function

In [119]:
def fetch_record(clsName = None, fetchColumn = [], searchBy = {}, offset_number = 1,  count_number = 6, PageNo = 1, total_pages=0):
    records = []
    rc = []
    try:
        if not len(fetchColumn):
            q = session.query(clsName)
        else:
            q = session.query(*[getattr(clsName, attr) for attr in fetchColumn])

        for k, v in searchBy.items():
            f = getattr(clsName,k)
            q = q.filter(f == v)
        records = [r._asdict() for r in q.offset(offset_number).limit(count_number).all()]
    except Exception as e:
        print(e)
        session.rollback()
        
    return records

cls    = IpStatus
f      = ['Ip','Status']
s      = {"Status":"Free"}
ot     = 0
nr     = 1
pg     = 1
tp     = 0

reco = fetch_record(cls,f,s,ot,nr,pg,tp)
print(reco)


[]


# 3. U - From CURD Operation

In [133]:
def update_records(clsName = None, searchBy = {}, updateTo={}):
    try:
        q = session.query(clsName)
        for k, v in searchBy.items():
            f = getattr(clsName,k)
            q = q.filter(f == v)
        print(q)
        q.update(updateTo)
        session.commit()
    except Exception as e:
        print(e)
        session.rollback()
    return "passs"
    
c = IpStatus
f = {"City":"NVMB"}
u = {"Status" : "Used","Remark":"boked"}

rec = update_records(c,f,u)

SELECT t_ipstatus."Ip" AS "t_ipstatus_Ip", t_ipstatus."Status" AS "t_ipstatus_Status", t_ipstatus."City" AS "t_ipstatus_City", t_ipstatus."ServerHall" AS "t_ipstatus_ServerHall", t_ipstatus."ServerHallId" AS "t_ipstatus_ServerHallId", t_ipstatus."Env" AS "t_ipstatus_Env", t_ipstatus."RackNo" AS "t_ipstatus_RackNo", t_ipstatus."Remark" AS "t_ipstatus_Remark" 
FROM t_ipstatus 
WHERE t_ipstatus."City" = ?


# 4. D - From Delete Operation

In [135]:
def delete_records(clsName = None, searchBy = {}):
    try:
        q = session.query(clsName)
        for k, v in searchBy.items():
            f = getattr(clsName,k)
            q = q.filter(f == v)
        print(q)
        
        q.delete(synchronize_session=False)
        session.commit()
    except Exception as e:
        print(e)
        session.rollback()
    return "passs"
    
c = IpStatus
f = {"City":"NVMB"}

rec = delete_records(c,f)

SELECT t_ipstatus."Ip" AS "t_ipstatus_Ip", t_ipstatus."Status" AS "t_ipstatus_Status", t_ipstatus."City" AS "t_ipstatus_City", t_ipstatus."ServerHall" AS "t_ipstatus_ServerHall", t_ipstatus."ServerHallId" AS "t_ipstatus_ServerHallId", t_ipstatus."Env" AS "t_ipstatus_Env", t_ipstatus."RackNo" AS "t_ipstatus_RackNo", t_ipstatus."Remark" AS "t_ipstatus_Remark" 
FROM t_ipstatus 
WHERE t_ipstatus."City" = ?
